# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770871557576                   -0.53    8.0         
  2   -2.772125362034       -2.90       -1.30    1.0    210ms
  3   -2.772168804953       -4.36       -2.69    1.0    177ms
  4   -2.772170705083       -5.72       -3.49    2.0    274ms
  5   -2.772170719207       -7.85       -3.75    2.0    202ms
  6   -2.772170722932       -8.43       -4.62    1.0    183ms
  7   -2.772170723003      -10.15       -5.06    1.0    206ms
  8   -2.772170723015      -10.92       -6.07    2.0    216ms
  9   -2.772170723015   +  -13.41       -6.02    2.0    241ms
 10   -2.772170723015      -13.00       -6.66    1.0    194ms
 11   -2.772170723015      -13.85       -7.32    1.0    220ms
 12   -2.772170723015      -14.07       -8.13    2.0    219ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.773557963930991

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770747248134                   -0.52    9.0         
  2   -2.772059667013       -2.88       -1.32    1.0    232ms
  3   -2.772082939868       -4.63       -2.43    1.0    173ms
  4   -2.772083335027       -6.40       -3.12    1.0    235ms
  5   -2.772083417732       -7.08       -4.49    2.0    203ms
  6   -2.772083417782      -10.30       -4.62    1.0    212ms
  7   -2.772083417807      -10.60       -5.59    1.0    185ms
  8   -2.772083417811      -11.47       -7.00    2.0    221ms
  9   -2.772083417811   +  -14.75       -7.00    2.0    220ms
 10   -2.772083417811      -14.45       -7.69    1.0    208ms
 11   -2.772083417811   +  -14.75       -9.06    2.0    224ms

Polarizability via ForwardDiff:       1.772534973058285
Polarizability via finite difference: 1.773557963930991
